In [ ]:
# @title Preprocess

train_model = "yolov8s.yaml"
train_epochs = 50
train_batchsize = 63
train_imagesize = 256
train_optimizer = 'Adam'

%cp /content/drive/MyDrive/dataaugment_dataset.zip /content/mydata.zip
%cd /content
!unzip mydata.zip
%rm /content/mydata.zip
%cd /content
!git clone https://github.com/ultralytics/ultralytics.git

Albumentations를 사용하기 위해 augment.py 파일을 수정해야 합니다. augment.py 파일은 /ultralytics/ultralytics/data/augment.py 경로에 위치해 있습니다.

해당 파일의 819번 줄을 보면 Albumentations 클래스가 정의되어 있을 것입니다. 해당 클래스의 `__init__` 함수의 T 배열 정의 부분을 수정하면 사용할 데이터 증강 기법들을 직접 지정할 수 있습니다.

해당 부분을 다음 코드로 교체합니다:

```python
T = [
  A.CLAHE(p=0.2),
  A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.4, hue=0.05, p=0.2),
  A.Downscale(scale_min=0.25, scale_max=0.5, p=0.2),
  A.GaussNoise(var_limit=(20, 60), p=0.2),
  A.ImageCompression(quality_lower=50, quality_upper=90, p=0.2),
  A.ISONoise(p=0.2),
  A.MultiplicativeNoise(per_channel=True, p=0.2),
  A.RandomFog(p=0.2),
  A.RandomGamma(p=0.2),
  A.RandomBrightnessContrast(p=0.2),
  A.Sharpen(p=0.2),
  A.Blur(p=0.2),
  A.MedianBlur(p=0.2)
]
```

In [2]:
# @title Run (must execute this after editing augment.py)
%cd /content/ultralytics
%pip install -e .

from ultralytics import YOLO
model = YOLO(train_model)
# see https://docs.ultralytics.com/usage/cfg/#train
model.train(
  name="s-rotcrop-colormod-noise-high",
  data="/content/datas/rotcrop-colormod-noise.yaml",
  epochs=train_epochs,
  imgsz=train_imagesize,
  batch=train_batchsize,
  optimizer=train_optimizer,
  cache=True,
  hsv_h=0.05,
  hsv_s=0.5,
  hsv_v=0.2,
  degrees=10,
  translate=0.1,
  scale=0.2,
  shear=5,
  perspective=0,
  flipud=0.2,
  fliplr=0.5,
  mosaic=1.0,
  mixup=0)
model.val()

import locale
locale.getpreferredencoding = lambda: "UTF-8"

%cd /content/ultralytics/runs/detect
!zip -r -1 archive.zip *
%cp -rf archive.zip /content/drive/MyDrive/augm_rotcrop_colormod_noise_high-small.zip

Ultralytics YOLOv8.1.5 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=/content/datas/rotcrop-colormod-noise.yaml, epochs=50, time=None, patience=50, batch=63, imgsz=256, save=True, save_period=-1, cache=True, device=None, workers=8, project=None, name=s-rotcrop-colormod-noise-high, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, 

100%|██████████| 755k/755k [00:00<00:00, 24.3MB/s]


Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 106MB/s]


AMP: checks passed ✅


...


50 epochs completed in 0.269 hours.
Optimizer stripped from /content/ultralytics/runs/detect/s-rotcrop-colormod-noise-high/weights/last.pt, 22.5MB
Optimizer stripped from /content/ultralytics/runs/detect/s-rotcrop-colormod-noise-high/weights/best.pt, 22.5MB

Validating /content/ultralytics/runs/detect/s-rotcrop-colormod-noise-high/weights/best.pt...
Ultralytics YOLOv8.1.5 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s summary (fused): 168 layers, 11127519 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  4.45it/s]


                   all         60         60      0.942      0.984      0.968      0.729
            box_target         60         11      0.826          1      0.884      0.696
       cylinder_target         60         12      0.992          1      0.995      0.733
  cylinder_target_body         60          5      0.943          1      0.995      0.838
  cylinder_target_head         60          5      0.949          1      0.995      0.715
         sphere_target         60         27          1      0.919      0.968      0.665
Speed: 0.0ms preprocess, 1.1ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to /content/ultralytics/runs/detect/s-rotcrop-colormod-noise-high
Ultralytics YOLOv8.1.5 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s summary (fused): 168 layers, 11127519 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /content/datasets/rotcrop-colormod-noise/valid/labels.cache... 60 images, 22 backgrounds, 0 corrupt: 100%|██████████| 60/60 [00:00<?, ?it/s]
val: Caching images (0.0GB True): 100%|██████████| 60/60 [00:00<00:00, 575.22it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.36it/s]


                   all         60         60      0.942      0.984      0.968      0.728
            box_target         60         11      0.826          1      0.884      0.696
       cylinder_target         60         12      0.992          1      0.995      0.733
  cylinder_target_body         60          5      0.943          1      0.995      0.833
  cylinder_target_head         60          5      0.949          1      0.995      0.715
         sphere_target         60         27          1      0.919      0.968      0.665
Speed: 0.0ms preprocess, 5.3ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to /content/ultralytics/runs/detect/s-rotcrop-colormod-noise-high2
/content/ultralytics/runs/detect
  adding: s-rotcrop-colormod-noise-high/ (stored 0%)
  adding: s-rotcrop-colormod-noise-high/val_batch0_pred.jpg (deflated 17%)
  adding: s-rotcrop-colormod-noise-high/train_batch1000.jpg (deflated 4%)
  adding: s-rotcrop-colormod-noise-high/confusion_matrix.png (deflat